In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

In [ ]:
# List of URLs from the producer's site
list_of_oils = ['https://oleofarm.pl/produkt/olej-konopny-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-krokoszowy-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-kukurydziany-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-lniany-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-slonecznikowy-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-sezamowy-nierafinowany/',
'https://oleofarm.pl/produkt/olej-z-awokado-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-z-orzechow-laskowych-nierafinowany/',
'https://oleofarm.pl/produkt/olej-z-orzechow-wloskich-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-z-pestek-dyni-tloczony-na-zimno/',
'https://oleofarm.pl/produkt/olej-z-lnianki/',
'https://oleofarm.pl/produkt/olej-z-czarnuszki/',
'https://oleofarm.pl/produkt/olej-ryzowy-rafinowany/',
'https://oleofarm.pl/produkt/olej-arachidowy-nierafinowany/'
]

In [ ]:
# initiating empty DataFrame for further populating with data
frame = pd.DataFrame(columns=['Features','Details','Product'])

# Looping over the list of URLs to scrape table with nutrition facts
for oil in list_of_oils:
    # get html from page
    r = requests.get(oil)
    # wait
    time.sleep(5)
    # turn text to bs object
    soup = BeautifulSoup(r.text)
    # create dataframe based on table from html
    df = pd.read_html(str(soup.find("table", {"class": ""})))
    df = df[0].rename(columns={0:"Features",1:"Details"})
    # extract name of the product
    df['Product'] = re.search('produkt/([^/]*)/',oil).group(1).replace('-',' ').title()
    frame = pd.concat([frame,df])

In [ ]:
# Saving dataframe to CSV
frame.to_csv('oils.csv')

In [7]:
# Getting missing oil from different website

# sending request
r1 = requests.get('https://aptekazawiszy.pl/olej-z-ostropestu-tloczony-na-zimno-250-ml-oleofarm.html')
# turning response to bs object
soup1 = BeautifulSoup(r1.text)

# finding div with nutrition facts
div_apteka = soup1.select('div.field:nth-child(4) > div:nth-child(2)')

# preparing empty dataframe to append extract data
ostropest_frame = pd.DataFrame(columns=['Data'])

# looping over all p tags to extract text and append to dataframe
for x in div_apteka[0].find_all('p'):
    # print(x.text)
    ostropest_frame = ostropest_frame.append({'Data':x.text},ignore_index=True)

# leaving only relevant data
ostropest_frame = ostropest_frame[2:24]

# renaming column and addition additional info
ostropest_frame = ostropest_frame.rename(columns={'Data':"Features"})
ostropest_frame['Details'] = ostropest_frame['Features']
ostropest_frame['Product'] = 'Olej Z Ostropestu Tloczony Na Zimno'

ostropest_frame = ostropest_frame.reset_index(drop=True)

# removing empty data and shifting up the rows so that naming of nutrition elements are aligned with their values
ostropest_frame.iloc[2:20,1] = ostropest_frame.iloc[2:20,1].shift(-1)

ostropest_frame = ostropest_frame.drop([3,5,7,9,11,13,15,17,19])



In [14]:
# extracting data using regex
ostropest_frame = ostropest_frame.drop(1)

ostropest_frame.iloc[10,0] = re.search('.+?\)',ostropest_frame.iloc[10,0]).group(0)

ostropest_frame.iloc[11,0] = re.search('.+?\)',ostropest_frame.iloc[11,0]).group(0)

ostropest_frame.iloc[10,1] = re.search('\)\s(.+)',ostropest_frame.iloc[10,1]).group(1)

ostropest_frame.iloc[11,1] = re.search('\)\s(.+)',ostropest_frame.iloc[11,1]).group(1)

ostropest_frame.iloc[0,1] = re.search('\d\d\d ml',ostropest_frame.iloc[0,1]).group(0)

ostropest_frame.iloc[0,0] = re.search('\w*\s\w*',ostropest_frame.iloc[0,0]).group(0)

# saving to csv
ostropest_frame.to_csv('last oil.csv')


In [12]:
# getting data for argan oil which we want to compare with others
r2 = requests.get('https://www.drmax.pl/oleofarm-olej-arganowy-100-ml-100020367')

# turning fetched HTML to bs object and loading table to pandas
soup3 = BeautifulSoup(r2.text)
df3 = pd.read_html(str(soup3.find("table")))

# renaming columns and adding one
df3 = df3[0].rename(columns={0:"Features",1:"Details"})

df3['Product'] = 'Olej arganowy nierafinowny'.title()

AttributeError: 'NoneType' object has no attribute 'group'

In [13]:
# concatenating frames into one
all_oils = pd.concat([df,ostropest_frame,df3]).drop(columns=['Unnamed: 0'])

# saving to csv but now without index
all_oils.to_csv('all_oils.csv',index=False)

,Features,Details,Product
0,Wartość odżywcza 100 ml:,Wartość odżywcza 100 ml:,Olej Z Ostropestu Tloczony Na Zimno
1,,,Olej Z Ostropestu Tloczony Na Zimno
2,Wartość energetyczna (energia),3378 kJ/822 kcal,Olej Z Ostropestu Tloczony Na Zimno
4,"Tłuszcz, w tym:",92 g,Olej Z Ostropestu Tloczony Na Zimno
6,Kwasy tłuszczowe nasycone,18 g,Olej Z Ostropestu Tloczony Na Zimno
8,Kwasy tłuszczowe jednonienasycone,25 g,Olej Z Ostropestu Tloczony Na Zimno
10,Kwasy tłuszczowe wielonienasycone,49 g,Olej Z Ostropestu Tloczony Na Zimno
12,"Węglowodany, w tym:",0 g,Olej Z Ostropestu Tloczony Na Zimno
14,cukry,0 g,Olej Z Ostropestu Tloczony Na Zimno
16,Białko,0 g,Olej Z Ostropestu Tloczony Na Zimno


In [43]:
# adjusting the text for selected rows
all_oils.loc[all_oils['Features'].str.contains('energetyczna'),'Features'] = 'Wartość energetyczna (energia)'

# resetting index
all_oils = all_oils.reset_index(drop=True)

# getting kilocalories from text and assigning to in column
all_oils.loc[all_oils['Features'].str.contains('energetyczna'),'Details'] = (all_oils.loc[all_oils['Features'].str.contains('energetyczna'),'Details'].str.extract(r'/(\d\d\d)').astype(int)).squeeze(axis=1)

# adjusting text for selected rows by replacing text
all_oils['Features'] = all_oils['Features'].replace('Wartość energetyczna (energia)','Wartość energetyczna (energia) w kcal')

# removing unnecessary rows
final_oils = all_oils[~all_oils['Features'].str.contains('dżywc')]

# Adjusting text in rows either by replacing values or extracting text via regular expressions
final_oils.loc[final_oils['Features'].str.contains('Węglowodany,'),'Features'] = 'Węglowodany'

final_oils.loc[final_oils['Features'].str.contains('Tłuszcz,'),'Features'] = 'Tłuszcz'

final_oils.loc[final_oils['Features'].str.contains('kwasy'),'Features'] = final_oils.loc[final_oils['Features'].str.contains('kwasy'),'Features'].str.extract(r'(\w+\s?\w+\s+\w+)').squeeze(axis=1).str.title().str.replace("  "," ")

final_oils.loc[final_oils['Features'].str.contains('ukry'),'Features'] = final_oils.loc[final_oils['Features'].str.contains('ukry'),'Features'].str.extract(r'(\w+)').squeeze(axis=1).str.title()

# getting numbers to floats
final_oils['Details'] = final_oils['Details'].astype(str).str.replace(",",'.')

final_oils['Details'] = final_oils['Details'].str.extract(r"(\d+.?\d?)").astype(float)

# cleaning wrong entries
final_oils.replace({'Omega – 6 (kwas linolowy)':"Omega-6 (kwas linolowy)",'Omega – 9 (kwas oleinowy)':"Omega-9 (kwas oleinowy)",'Omega-9 (kwas oleinowy':'Omega-9 (kwas oleinowy)'})
final_oils[(final_oils['Product'].isin(['Olej Z Lnianki'])) &(final_oils['Features'].str.contains('6'))]['Features'] = 'Omega-6 (kwas linolowy)'

# saving clean data to csv
final_oils.to_csv('clean_oils_data.csv',index=False)

c:\Users\Maciek\PycharmProjects\Python\venv\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\Maciek\PycharmProjects\Python\venv\lib\site-packages\pandas\core\indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
